# 🎴 Card Recognition V5 - With Confusion Logging

**Curriculum Learning + Live Accuracy + Confusion Tracking**

- See which cards the model confuses during training
- Uses JSON for actual card names
- Phases: Light → Medium → Heavy augmentation

---

## 1️⃣ Setup

In [ ]:
!nvidia-smi
import torch
print(f"PyTorch: {torch.__version__}, CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
!pip install -q timm albumentations opencv-python-headless tqdm imagehash

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2️⃣ Load Data & Card JSON

In [ ]:
import os, zipfile, json, random
from pathlib import Path
from PIL import Image, ImageOps
from tqdm.notebook import tqdm
from collections import Counter
import numpy as np
import cv2

ZIP_PATH = "/content/drive/MyDrive/CardData/card_images.zip"
IMAGE_DIR = "/content/card_images"
CHECKPOINT_DIR = '/content/checkpoints'
DRIVE_OUTPUT = '/content/drive/MyDrive/CardRecognition_Models'
CARD_JSON = '/content/drive/MyDrive/CardData/card-flattened-with-phash.json'

for d in [CHECKPOINT_DIR, DRIVE_OUTPUT]:
    os.makedirs(d, exist_ok=True)

# Extract images
if os.path.exists(f"{IMAGE_DIR}/.extracted"):
    print(f"✓ Already extracted")
elif os.path.exists(ZIP_PATH):
    print("Extracting...")
    !rm -rf {IMAGE_DIR}
    os.makedirs(IMAGE_DIR, exist_ok=True)
    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        z.extractall(IMAGE_DIR)
    Path(f"{IMAGE_DIR}/.extracted").touch()
    print(f"✓ Done")

In [ ]:
# Validate images
if os.path.exists(f"{IMAGE_DIR}/.validated"):
    print("✓ Already validated")
else:
    print("Validating...")
    corrupted = []
    for p in tqdm(list(Path(IMAGE_DIR).glob('*'))):
        if p.suffix.lower() in ['.jpg','.jpeg','.png','.webp']:
            try:
                with Image.open(p) as img: img.verify()
                with Image.open(p) as img: img.load()
            except:
                corrupted.append(p.name)
                p.unlink()
    Path(f"{IMAGE_DIR}/.validated").touch()
    print(f"✓ Removed {len(corrupted)} corrupted")

In [ ]:
# Load card JSON for names
with open(CARD_JSON, 'r') as f:
    all_cards = json.load(f)

# Create lookup: printing_unique_id -> card info
card_lookup = {c['printing_unique_id']: c for c in all_cards}

def get_card_name(printing_id):
    """Get human-readable card name from printing ID."""
    card = card_lookup.get(printing_id, {})
    name = card.get('name', printing_id[:20])
    foil = card.get('foiling', '')
    if foil and foil != 'S':  # S = Standard/non-foil
        return f"{name} ({foil})"
    return name

print(f"✓ Loaded {len(all_cards):,} cards from JSON")
print(f"   Example: {get_card_name(list(card_lookup.keys())[0])}")

## 3️⃣ Curriculum Augmentations

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

def get_light_aug(size=224):
    return A.Compose([
        A.Resize(size, size),
        A.Rotate(limit=3, p=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3),
        A.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
        ToTensorV2()
    ])

def get_medium_aug(size=224):
    return A.Compose([
        A.Resize(size, size),
        A.Perspective(scale=(0.02, 0.05), p=0.3),
        A.Rotate(limit=8, p=0.4),
        A.OneOf([A.GaussianBlur(blur_limit=3), A.MotionBlur(blur_limit=3)], p=0.2),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.4),
        A.GaussNoise(var_limit=(5, 20), p=0.2),
        A.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
        ToTensorV2()
    ])

def get_heavy_aug(size=224):
    return A.Compose([
        A.Resize(size, size),
        A.Perspective(scale=(0.02, 0.08), p=0.5),
        A.Rotate(limit=15, border_mode=cv2.BORDER_CONSTANT, p=0.5),
        A.OneOf([A.MotionBlur(blur_limit=5), A.GaussianBlur(blur_limit=5)], p=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25, p=0.5),
        A.RandomShadow(shadow_roi=(0, 0.3, 1, 1), p=0.2),
        A.GaussNoise(var_limit=(10, 40), p=0.3),
        A.ImageCompression(quality_lower=70, p=0.2),
        A.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
        ToTensorV2()
    ])

def get_val_transforms(size=224):
    return A.Compose([
        A.Resize(size, size),
        A.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
        ToTensorV2()
    ])

print("✓ Augmentations ready")

In [ ]:
class SyntheticBackground:
    def __init__(self, output_size=(480, 640)):
        self.output_size = output_size
    
    def get_random_bg(self):
        if random.random() > 0.3:
            color = tuple(random.randint(0, 255) for _ in range(3))
            return np.full((*self.output_size, 3), color, dtype=np.uint8)
        else:
            c1, c2 = np.random.randint(0, 255, 3), np.random.randint(0, 255, 3)
            arr = np.zeros((*self.output_size, 3), dtype=np.uint8)
            for i in range(self.output_size[0]):
                t = i / self.output_size[0]
                arr[i] = (c1 * (1-t) + c2 * t).astype(np.uint8)
            return arr
    
    def composite(self, card_img):
        bg = self.get_random_bg()
        h, w = card_img.shape[:2]
        scale = random.uniform(0.5, 0.8)
        new_h, new_w = int(self.output_size[0]*scale), int(self.output_size[0]*scale*w/h)
        card = cv2.resize(card_img, (new_w, new_h))
        y = random.randint(0, max(0, self.output_size[0]-new_h))
        x = random.randint(0, max(0, self.output_size[1]-new_w))
        bg[y:y+new_h, x:x+new_w] = card[:min(new_h, self.output_size[0]-y), :min(new_w, self.output_size[1]-x)]
        return bg

synth_bg = SyntheticBackground()
print("✓ SyntheticBackground ready")

## 4️⃣ Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm

class GeM(nn.Module):
    def __init__(self, p=3.0):
        super().__init__()
        self.p = nn.Parameter(torch.ones(1) * p)
    def forward(self, x):
        return F.adaptive_avg_pool2d(x.clamp(min=1e-6).pow(self.p), 1).pow(1./self.p).view(x.size(0), -1)

class ColorBranch(nn.Module):
    def __init__(self, bins=32, out_dim=64):
        super().__init__()
        self.bins = bins
        self.fc = nn.Sequential(nn.Linear(bins*3, 128), nn.ReLU(), nn.Dropout(0.3), nn.Linear(128, out_dim))
        self.register_buffer('mean', torch.tensor([0.485,0.456,0.406]).view(1,3,1,1))
        self.register_buffer('std', torch.tensor([0.229,0.224,0.225]).view(1,3,1,1))
    
    def forward(self, x):
        x_denorm = ((x * self.std + self.mean) * 255).clamp(0, 255)
        hists = []
        for i in range(x.shape[0]):
            img = x_denorm[i].permute(1,2,0).cpu().numpy().astype(np.uint8)
            hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
            h = np.histogram(hsv[:,:,0], bins=self.bins, range=(0,180))[0]
            s = np.histogram(hsv[:,:,1], bins=self.bins, range=(0,256))[0]
            v = np.histogram(hsv[:,:,2], bins=self.bins, range=(0,256))[0]
            hist = np.concatenate([h,s,v]).astype(np.float32)
            hists.append(hist / (hist.sum() + 1e-8))
        return self.fc(torch.tensor(np.stack(hists), device=x.device, dtype=torch.float32))

class CardNet(nn.Module):
    def __init__(self, emb_dim=512, color_dim=64):
        super().__init__()
        self.backbone = timm.create_model('mobilenetv3_small_100', pretrained=True, num_classes=0, global_pool='')
        with torch.no_grad():
            self.n_feat = self.backbone(torch.randn(1,3,224,224)).shape[1]
        self.gem = GeM()
        self.color = ColorBranch(out_dim=color_dim)
        self.fc = nn.Linear(self.n_feat + color_dim, emb_dim)
        self.bn = nn.BatchNorm1d(emb_dim)
        self.drop = nn.Dropout(0.5)
    
    def forward(self, x):
        v = self.gem(self.backbone(x))
        c = self.color(x)
        return F.normalize(self.drop(self.bn(self.fc(torch.cat([v, c], dim=1)))), p=2, dim=1)

print("✓ CardNet ready")

## 5️⃣ Dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CardDataset(Dataset):
    def __init__(self, image_dir, rotations=[0, 90, 180, 270]):
        self.images = sorted([f for f in Path(image_dir).iterdir() 
                              if f.suffix.lower() in ['.jpg','.jpeg','.png','.webp']])
        self.samples = [(i, r) for i in range(len(self.images)) for r in rotations]
        self.transform = get_light_aug()
        self.use_bg = False
        self.synth_bg = SyntheticBackground()
        print(f"Dataset: {len(self.images)} cards × {len(rotations)} rot = {len(self.samples)} samples")
    
    def set_phase(self, phase):
        if phase == 1:
            self.transform, self.use_bg = get_light_aug(), False
            print("📚 Phase 1: Light augmentation")
        elif phase == 2:
            self.transform, self.use_bg = get_medium_aug(), False
            print("📚 Phase 2: Medium augmentation")
        else:
            self.transform, self.use_bg = get_heavy_aug(), True
            print("📚 Phase 3: Heavy + backgrounds")
    
    def __len__(self): return len(self.samples)
    
    def __getitem__(self, idx):
        img_idx, rot = self.samples[idx]
        try:
            img = np.array(Image.open(self.images[img_idx]).convert('RGB'))
            if rot == 90: img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            elif rot == 180: img = cv2.rotate(img, cv2.ROTATE_180)
            elif rot == 270: img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
            if self.use_bg and random.random() > 0.5:
                img = self.synth_bg.composite(img)
            return self.transform(image=img)['image'], img_idx
        except:
            return self.__getitem__(random.randint(0, len(self.samples)-1))
    
    def get_num_classes(self): return len(self.images)
    def get_printing_id(self, idx): return self.images[idx].stem

print("✓ CardDataset ready")

## 6️⃣ Confusion Tracker

In [ ]:
class ConfusionTracker:
    """Track which cards get confused during training."""
    
    def __init__(self, get_name_fn):
        self.confusions = Counter()  # (actual, predicted) -> count
        self.get_name = get_name_fn
    
    def log_confusion(self, actual_id, predicted_id):
        """Log a confusion between two cards."""
        if actual_id != predicted_id:
            actual_name = self.get_name(actual_id)
            pred_name = self.get_name(predicted_id)
            self.confusions[(actual_name, pred_name)] += 1
    
    def get_top_confusions(self, n=10):
        """Get most common confusion pairs."""
        return self.confusions.most_common(n)
    
    def print_report(self, n=10):
        """Print confusion report."""
        top = self.get_top_confusions(n)
        if not top:
            print("   No confusions logged yet!")
            return
        
        print(f"\n   🔄 TOP {min(n, len(top))} CONFUSIONS:")
        for (actual, pred), count in top:
            print(f"      '{actual}' → '{pred}' ({count}x)")
    
    def reset(self):
        self.confusions.clear()

print("✓ ConfusionTracker ready")

In [ ]:
class AccuracyMonitor:
    """Monitor accuracy and track confusions."""
    
    def __init__(self, model, dataset, device, confusion_tracker, n_test=100):
        self.model = model
        self.dataset = dataset
        self.device = device
        self.tracker = confusion_tracker
        self.n_test = n_test
        self.transform = get_val_transforms()
        self.ref_embeddings = None
        self.ref_ids = None
    
    def build_references(self):
        self.model.eval()
        embeddings, ids = [], []
        with torch.no_grad():
            for img_path in self.dataset.images:
                try:
                    img = np.array(Image.open(img_path).convert('RGB'))
                    emb = self.model(self.transform(image=img)['image'].unsqueeze(0).to(self.device))
                    embeddings.append(emb.cpu())
                    ids.append(img_path.stem)
                except: pass
        self.ref_embeddings = torch.cat(embeddings, dim=0)
        self.ref_ids = ids
    
    def compute_accuracy(self):
        if self.ref_embeddings is None:
            self.build_references()
        
        self.model.eval()
        self.tracker.reset()  # Reset confusions for this check
        
        test_indices = random.sample(range(len(self.dataset.images)), 
                                      min(self.n_test, len(self.dataset.images)))
        top1, top5, top10 = 0, 0, 0
        
        with torch.no_grad():
            for idx in test_indices:
                img_path = self.dataset.images[idx]
                actual_id = img_path.stem
                
                try:
                    img = np.array(Image.open(img_path).convert('RGB'))
                    query = self.model(self.transform(image=img)['image'].unsqueeze(0).to(self.device)).cpu()
                    sims = F.cosine_similarity(query, self.ref_embeddings)
                    top_idx = sims.argsort(descending=True)[:10]
                    top_ids = [self.ref_ids[i] for i in top_idx]
                    
                    # Track accuracy
                    if actual_id == top_ids[0]: top1 += 1
                    if actual_id in top_ids[:5]: top5 += 1
                    if actual_id in top_ids[:10]: top10 += 1
                    
                    # Log confusion if wrong
                    if actual_id != top_ids[0]:
                        self.tracker.log_confusion(actual_id, top_ids[0])
                except: pass
        
        n = len(test_indices)
        return {'top1': 100*top1/n, 'top5': 100*top5/n, 'top10': 100*top10/n}

print("✓ AccuracyMonitor ready")

## 7️⃣ Training Setup

In [ ]:
class CosFaceLoss(nn.Module):
    def __init__(self, num_classes, emb_dim, scale=64.0, margin=0.4):
        super().__init__()
        self.scale, self.margin = scale, margin
        self.weight = nn.Parameter(torch.FloatTensor(num_classes, emb_dim))
        nn.init.xavier_uniform_(self.weight)
    
    def forward(self, emb, labels):
        W = F.normalize(self.weight, p=2, dim=1)
        cos = F.linear(emb, W)
        one_hot = torch.zeros_like(cos).scatter_(1, labels.view(-1,1), 1.0)
        return F.cross_entropy((cos - one_hot * self.margin) * self.scale, labels)

PHASES = {1: (1, 10), 2: (11, 20), 3: (21, 100)}

CONFIG = {
    'epochs': 60,
    'batch_size': 64,
    'lr': 1e-3,
    'weight_decay': 1e-4,
    'emb_dim': 512,
    'patience': 15,
    'unfreeze_epoch': 8,
    'check_interval': 5,
}
print("✓ Config:", CONFIG)

In [ ]:
# Create dataset and loaders
train_ds = CardDataset(IMAGE_DIR)
val_ds = CardDataset(IMAGE_DIR, rotations=[0])
val_ds.transform = get_val_transforms()
val_ds.use_bg = False

indices = np.random.permutation(len(train_ds.images))
split = int(0.85 * len(train_ds.images))
train_idx, val_idx = set(indices[:split]), set(indices[split:])

train_samples = [i for i, (c, _) in enumerate(train_ds.samples) if c in train_idx]
val_samples = [i for i, (c, _) in enumerate(val_ds.samples) if c in val_idx]

train_loader = DataLoader(torch.utils.data.Subset(train_ds, train_samples),
                          batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2, 
                          pin_memory=True, drop_last=True)
val_loader = DataLoader(torch.utils.data.Subset(val_ds, val_samples),
                        batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2, pin_memory=True)

num_classes = train_ds.get_num_classes()
print(f"✓ Train: {len(train_samples):,} | Val: {len(val_samples):,} | Classes: {num_classes:,}")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CardNet(emb_dim=CONFIG['emb_dim']).to(device)

for p in model.backbone.parameters(): p.requires_grad = False
print("✓ Backbone frozen")

criterion = CosFaceLoss(num_classes, CONFIG['emb_dim']).to(device)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                              lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['epochs'])
scaler = torch.amp.GradScaler('cuda')

# Confusion tracker with card name lookup
confusion_tracker = ConfusionTracker(get_card_name)
accuracy_monitor = AccuracyMonitor(model, train_ds, device, confusion_tracker, n_test=100)

print("✓ Ready")

## 8️⃣ Training with Confusion Logging

In [ ]:
best_loss = float('inf')
patience_counter = 0
history = {'train': [], 'val': [], 'top1': [], 'top5': []}
current_phase = 0
RESUME_PATH = f"{CHECKPOINT_DIR}/best_model.pth"

print("="*70)
print("TRAINING WITH CONFUSION LOGGING")
print("="*70)

for epoch in range(1, CONFIG['epochs'] + 1):
    # Update curriculum phase
    for phase, (start, end) in PHASES.items():
        if start <= epoch <= end and phase != current_phase:
            current_phase = phase
            train_ds.set_phase(phase)
            accuracy_monitor.ref_embeddings = None  # Rebuild refs
            break
    
    # Unfreeze backbone
    if epoch == CONFIG['unfreeze_epoch']:
        print("\n🔓 Unfreezing backbone...")
        for p in model.backbone.parameters(): p.requires_grad = True
        optimizer = torch.optim.AdamW([
            {'params': model.backbone.parameters(), 'lr': CONFIG['lr']/10},
            {'params': model.gem.parameters()},
            {'params': model.color.parameters()},
            {'params': model.fc.parameters()},
            {'params': model.bn.parameters()},
        ], lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])
    
    # Training
    model.train()
    train_loss = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch} [P{current_phase}]", leave=False):
        images, labels = images.to(device), labels.to(device)
        with torch.amp.autocast('cuda'):
            loss = criterion(model(images), labels)
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            val_loss += criterion(model(images), labels).item()
    val_loss /= len(val_loader)
    
    scheduler.step()
    history['train'].append(train_loss)
    history['val'].append(val_loss)
    
    # Accuracy check with confusion logging
    if epoch % CONFIG['check_interval'] == 0:
        acc = accuracy_monitor.compute_accuracy()
        history['top1'].append(acc['top1'])
        history['top5'].append(acc['top5'])
        print(f"\nEpoch {epoch}: Loss={train_loss:.2f}/{val_loss:.2f} | "
              f"Top-1: {acc['top1']:.1f}% | Top-5: {acc['top5']:.1f}% | Top-10: {acc['top10']:.1f}%")
        confusion_tracker.print_report(n=5)
    else:
        print(f"Epoch {epoch}: Train={train_loss:.4f}, Val={val_loss:.4f}")
    
    # Save best
    if val_loss < best_loss:
        best_loss = val_loss
        patience_counter = 0
        torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),
                    'val_loss': val_loss, 'num_classes': num_classes}, RESUME_PATH)
        print("  💾 Saved")
    else:
        patience_counter += 1
        if patience_counter >= CONFIG['patience']:
            print("\n⚠️ Early stop!")
            break

print(f"\n✓ Done! Best: {best_loss:.4f}")

## 9️⃣ Results

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(history['train'], label='Train')
axes[0].plot(history['val'], label='Val')
axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('Loss'); axes[0].legend()
axes[0].set_title('Loss')

if history['top1']:
    x = list(range(CONFIG['check_interval'], len(history['top1'])*CONFIG['check_interval']+1, CONFIG['check_interval']))
    axes[1].plot(x, history['top1'], 'o-', label='Top-1')
    axes[1].plot(x, history['top5'], 's-', label='Top-5')
    axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('Accuracy %'); axes[1].legend()
    axes[1].set_title('Identification Accuracy')

plt.tight_layout()
plt.savefig(f"{CHECKPOINT_DIR}/training.png")
plt.show()

In [ ]:
import shutil
for f in ['best_model.pth', 'training.png']:
    src = f"{CHECKPOINT_DIR}/{f}"
    if os.path.exists(src):
        shutil.copy(src, DRIVE_OUTPUT)
        print(f"✓ Saved {f}")

## 🔟 Final Test

In [ ]:
print("Building final embeddings...")
ckpt = torch.load(RESUME_PATH)
model.load_state_dict(ckpt['model_state_dict'])
model.eval()

ref_embeddings, ref_names = [], []
with torch.no_grad():
    for img_path in tqdm(train_ds.images, desc="Building refs"):
        try:
            img = np.array(Image.open(img_path).convert('RGB'))
            emb = model(get_val_transforms()(image=img)['image'].unsqueeze(0).to(device))
            ref_embeddings.append(emb.cpu())
            ref_names.append(img_path.stem)
        except: pass

ref_embeddings = torch.cat(ref_embeddings, dim=0)
print(f"✓ {len(ref_embeddings):,} embeddings")

In [ ]:
# Final accuracy with confusion report
final_tracker = ConfusionTracker(get_card_name)
final_monitor = AccuracyMonitor(model, train_ds, device, final_tracker, n_test=200)
final_acc = final_monitor.compute_accuracy()

print(f"\n{'='*60}")
print(f"FINAL ACCURACY (200 test cards)")
print(f"{'='*60}")
print(f"Top-1:  {final_acc['top1']:.1f}%")
print(f"Top-5:  {final_acc['top5']:.1f}%")
print(f"Top-10: {final_acc['top10']:.1f}%")
final_tracker.print_report(n=10)

In [ ]:
# Test with upload
import imagehash
from google.colab import files

name_to_printings = {}
for c in all_cards:
    name = c.get('name', '')
    if name not in name_to_printings:
        name_to_printings[name] = []
    name_to_printings[name].append({'id': c['printing_unique_id'], 'set': c.get('set_id',''), 
                                    'foil': c.get('foiling',''), 'phash': c.get('image_phash','')[:64]})

def identify(img_path):
    pil = ImageOps.autocontrast(Image.open(img_path).convert('RGB'), cutoff=1)
    arr = np.array(pil)
    with torch.no_grad():
        q = model(get_val_transforms()(image=arr)['image'].unsqueeze(0).to(device)).cpu()
    sims = F.cosine_similarity(q, ref_embeddings)
    top_id = ref_names[sims.argmax().item()]
    name = get_card_name(top_id)
    conf = sims.max().item()
    
    # pHash for exact printing
    printings = name_to_printings.get(card_lookup.get(top_id, {}).get('name', ''), [])
    if printings:
        qh = imagehash.phash(pil, hash_size=16)
        best = min(printings, key=lambda p: (imagehash.hex_to_hash(p['phash'])-qh) if p['phash'] else 999)
        return {'name': name, 'conf': conf, 'set': best['set'], 'foil': best['foil']}
    return {'name': name, 'conf': conf}

print('Upload images to test:')
for f in files.upload().keys():
    r = identify(f)
    print(f"\n🎴 {r['name']} ({r['conf']*100:.0f}%)")
    print(f"   Set: {r.get('set','')} | Foil: {r.get('foil','')}")
    plt.imshow(Image.open(f)); plt.axis('off'); plt.show()

## ✅ Done!

**New: Confusion Logging** - See which cards the model struggles with!

Example output:
```
Epoch 10: Top-1: 75% | Top-5: 95%
   🔄 TOP 5 CONFUSIONS:
      'Razor Reflex (Red)' → 'Razor Reflex (Blue)' (3x)
      'Sink Below' → 'Sink Below (Yellow)' (2x)
```